In [1]:
from service.colpali_server import ColQwen2Service
from service.milvus_retriever import MilvusColbertRetriever
from pymilvus import MilvusClient
import os
# Get model service
print("---------------------Get model service---------------------")
base_model_path = "/home/linux/yyj/colpali/finetune/colqwen2-v1.0-hf"
adapter_path = "/home/linux/yyj/colpali/finetune/wiky_city_zh_0702_lr2e4_colqwen"
max_pixels = 1100*28*28
model_service = ColQwen2Service(base_model_path, adapter_path, max_pixels)

test_pages_dir = "/home/linux/yyj/colpali/finetune/mmlm-rag/test_pages"
queries = [
    "阿布达比5月气温怎么样",
    "阿布达比12月降水量怎么样",
]
qs = model_service.process_queries(queries)
ds = model_service.process_images(test_pages_dir)

# Get milvus service
print("---------------------Get milvus service---------------------")
client = MilvusClient(uri="http://localhost:19530")
# before inserting data, collection must be loaded in attu
retriever = MilvusColbertRetriever(collection_name="colqwen_test", milvus_client=client)
retriever.create_collection()
retriever.create_index()
print("--------------------Insert doc embeddings to milvus---------------------------")
filepaths = [os.path.join(test_pages_dir, name) for name in os.listdir(test_pages_dir)]
for i in range(len(filepaths)):
    data = {
        "colbert_vecs": ds[i].float().cpu().numpy(),
        "doc_id": i,
        "filepath": filepaths[i],
    }
    retriever.insert(data)
print("--------------------Test service---------------------------")
for query in qs:
    # query: 22*128
    query = query.float().cpu().numpy()
    result = retriever.search(query, topk=1)
    print(filepaths[result[0][1]])

/home/linux/anaconda3/envs/cp_ft/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You are attempting to use Flash Attention 2.0 without specifying a torch dtype. This might lead to unexpected behaviour


---------------------Get model service---------------------


100%|██████████| 3/3 [00:02<00:00,  1.07it/s]


---------------------Get milvus service---------------------
--------------------Insert doc embeddings to milvus---------------------------
--------------------Test service---------------------------
/home/linux/yyj/colpali/finetune/mmlm-rag/test_pages/test_page_001.png
/home/linux/yyj/colpali/finetune/mmlm-rag/test_pages/test_page_001.png


In [2]:
import os

pages_dir = "/home/linux/yyj/colpali/finetune/pdf2images_with_reference_per_city"
filepaths = []
for dir in os.listdir(pages_dir):
    dirpath = os.path.join(pages_dir, dir)
    for name in os.listdir(dirpath):
        filepaths.append(os.path.join(dirpath, name))
print(filepaths)

['/home/linux/yyj/colpali/finetune/pdf2images_with_reference_per_city/Edinburgh/page_003.png', '/home/linux/yyj/colpali/finetune/pdf2images_with_reference_per_city/Edinburgh/page_004.png', '/home/linux/yyj/colpali/finetune/pdf2images_with_reference_per_city/Edinburgh/page_009.png', '/home/linux/yyj/colpali/finetune/pdf2images_with_reference_per_city/Edinburgh/page_007.png', '/home/linux/yyj/colpali/finetune/pdf2images_with_reference_per_city/Edinburgh/page_006.png', '/home/linux/yyj/colpali/finetune/pdf2images_with_reference_per_city/Edinburgh/page_005.png', '/home/linux/yyj/colpali/finetune/pdf2images_with_reference_per_city/Edinburgh/page_001.png', '/home/linux/yyj/colpali/finetune/pdf2images_with_reference_per_city/Edinburgh/page_008.png', '/home/linux/yyj/colpali/finetune/pdf2images_with_reference_per_city/Edinburgh/page_002.png', '/home/linux/yyj/colpali/finetune/pdf2images_with_reference_per_city/Los_Angeles/page_010.png', '/home/linux/yyj/colpali/finetune/pdf2images_with_referen